In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

In [3]:
from coolprompt.assistant import PromptTuner
tuner = PromptTuner()

INFO 05-29 15:27:50 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-29 15:27:50 [__init__.py:239] Automatically detected platform cuda.
WARNING 05-29 15:27:50 [cuda.py:409] Detected different devices in the system: NVIDIA GeForce RTX 2080 Ti, NVIDIA TITAN RTX. Please make sure to set `CUDA_DEVICE_ORDER=PCI_BUS_ID` to avoid unexpected behavior.
WARNING 05-29 15:27:53 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-29 15:28:02 [config.py:717] This model supports multiple tasks: {'embed', 'reward', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 05-29 15:28:02 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-29 15:28:02 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_co

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 05-29 15:28:14 [loader.py:458] Loading weights took 8.38 seconds
INFO 05-29 15:28:14 [model_runner.py:1140] Model loading took 14.2427 GiB and 9.304607 seconds
INFO 05-29 15:28:20 [worker.py:287] Memory profiling takes 5.20 seconds
INFO 05-29 15:28:20 [worker.py:287] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.90) = 21.27GiB
INFO 05-29 15:28:20 [worker.py:287] model weights take 14.24GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 2.62GiB.
INFO 05-29 15:28:20 [executor_base.py:112] # cuda blocks: 3068, # CPU blocks: 4681
INFO 05-29 15:28:20 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 1.50x
INFO 05-29 15:28:25 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-29 15:28:43 [model_runner.py:1592] Graph capturing finished in 18 secs, took 0.21 GiB
INFO 05-29 15:28:43 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 28.93 seconds


In [11]:
from datasets import load_dataset

sst2 = load_dataset("sst2")
n_instances = 100
dataset = sst2['train']['sentence'][:n_instances]
targets = sst2['train']['label'][:n_instances]

start_prompt = '''
Please perform Sentiment Classification task.

Answer using the label from [0, 1], where 0 stands for negative, 1 stands for positive.
Generate the final answer bracketed with <ans> and </ans>.

The input:
'''

In [12]:
final_prompt = tuner.run(start_prompt=start_prompt,
                         dataset=dataset,
                         target=targets,
                         task='classification',
                         metric="accuracy")

Processed prompts:   0%|                      | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

Processed prompts:   0%|                    | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

WARNING 05-29 15:52:58 [scheduler.py:1768] Sequence group 288 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts:   0%|                    | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

WARNING 05-29 16:00:06 [scheduler.py:1768] Sequence group 388 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=101


In [13]:
print("PROMPT:", final_prompt)
print("INITIAL METRIC:", tuner.init_metric)
print("FINAL METRIC:", tuner.final_metric)

PROMPT: Perform Sentiment Classification task. Use the label [0, 1] where 0 represents negative and 1 represents positive. Generate the final answer in the format <ans> [0/1] </ans>. 
INITIAL METRIC: 0.41
FINAL METRIC: 0.0
